In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split


import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
pruebas = pd.read_csv('properati_dataset_modificado.csv')

In [15]:
sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

propiedades = pruebas
p = propiedades.loc[  (propiedades.superficie >= sup_max) | (propiedades.superficie <= sup_min) |\
                      (propiedades.lat >= lat_max) | (propiedades.lat <= lat_min) |\
                       (propiedades.lon >= lon_max) | (propiedades.lon <= lon_min),:]

columnas = ['superficie','lat', 'lon']

In [17]:
p.loc[:,columnas].describe()

,superficie,lat,lon
count,4.174000e+03,4174.000000,4174.000000
mean,1.143443e+03,-34.667990,-58.460701
std,5.025419e+04,0.347596,0.320485
min,1.000000e+00,-38.416097,-63.616672
25%,1.670000e+02,-34.756526,-58.632980
50%,2.245681e+02,-34.646440,-58.459830
75%,3.200000e+02,-34.598624,-58.381931
max,3.243242e+06,-29.424917,-57.558500


In [19]:
p.loc[:,columnas].count()

superficie    4174
lat           4174
lon           4174
dtype: int64

## Hay 4000 propiedades con datos "malos" en las pruebas a analizar

## Quizas por eso tenemos muy malos resultados